In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [150]:
!pip install strsimpy
from strsimpy.jaro_winkler import JaroWinkler
from strsimpy.damerau import Damerau
jarowinkler = JaroWinkler()
print(jarowinkler.similarity('My string', 'My tsring'))
print(jarowinkler.similarity('My string', 'My ntrisg'))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 45 kB 1.9 MB/s 
0.9740740740740741
0.8962962962962963


In [153]:
jarowinkler.similarity('name', 'name')

1.0

In [2]:
import re
import string 
import pandas as pd
import os.path
import json
from sklearn import metrics
import matplotlib.pyplot as plt
import time
import numpy as np
from collections import Counter
import string

In [4]:
df = pd.read_csv('/content/drive/MyDrive/THDL/newshop1.csv')
df = df.drop(columns = ['Unnamed: 0'])
df['publisher'] = df['publisher'].apply(lambda x: str(x).strip())
df.head()

,Unnamed: 0,author_name,cover_type,image_url,name,num_page,price,provider,publisher,release_year,size,url
0,0,Fuji,Mềm,https://newshop.vn/public/uploads/products/485...,Sách Tương Phản - Đồ Dùng Của Bé,20,60000,Nhà Sách AZ Việt Nam,Thế Giới,2022,150x150,https://newshop.vn/sach-tuong-phan-do-dung-cua...
1,1,Fuji,Cứng,https://newshop.vn/public/uploads/products/485...,Combo Sách Tương Phản (Bộ 4 Cuốn),20,240000,Nhà Sách AZ Việt Nam,Thế Giới,2022,150x150,https://newshop.vn/combo-sach-tuong-phan-bo-4-...
2,2,Mèo Vũ Trụ,Mềm,https://newshop.vn/public/uploads/products/485...,Chạm - Hành Trình Cảm Xúc Của 12 Chòm Sao,184,99000,Nhà Sách AZ Việt Nam,Thế Giới,2022,130x180,https://newshop.vn/cham-hanh-trinh-cam-xuc-cua...
3,3,Manh A Công Tử,Mềm,https://newshop.vn/public/uploads/products/485...,Môn Đồ - Sư Phụ Giả - Tập 7,224,129000,Nhà Sách AZ Việt Nam,Thế Giới,2022,145x205,https://newshop.vn/mon-do-su-phu-gia-tap-7.htm...
4,4,Motoro Mase,Mềm,https://newshop.vn/public/uploads/products/485...,Ikigami - Tuyển Tập Những Câu Chuyện Lay Động ...,216,45000,NXB Trẻ,Trẻ,2022,130x10,https://newshop.vn/ikigami-tuyen-tap-nhung-cau...


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25627 entries, 0 to 25626
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   author_name   25627 non-null  object
 1   cover_type    25627 non-null  object
 2   image_url     25627 non-null  object
 3   name          25627 non-null  object
 4   num_page      25627 non-null  object
 5   price         25627 non-null  int64 
 6   provider      25627 non-null  object
 7   publisher     25627 non-null  object
 8   release_year  25627 non-null  object
 9   size          25627 non-null  object
 10  url           25627 non-null  object
dtypes: int64(1), object(10)
memory usage: 2.2+ MB


In [72]:
def word_loglikelihood(w):
    w = w.lower()
    if w in V_freq:
        p_w_pos = (freq_pos.get(w,0)+1 / (N_pos+V))
        p_w_neg = (freq_neg.get(w,0)+1 / (N_neg+V)) 
        return np.log(p_w_pos/p_w_neg)
    else:
        return(0)


def text_loglikelihood(mytxt):
    mytxt = mytxt.lower().split(" ")
    score = logprior
    for w in mytxt:
        score+= word_loglikelihood(w)
        # print(w,word_loglikelihood(w))
    return(score)

In [117]:
columns_df = df.columns.tolist().copy()
columns_df.remove('url')
columns_df.remove('image_url')

In [119]:
columns_df

['author_name',
 'cover_type',
 'name',
 'num_page',
 'price',
 'provider',
 'publisher',
 'release_year',
 'size']

In [132]:
df_2['year'] = df_2['year'].apply(lambda x:str(x).split('.')[0])
df_2['year']

,author,cover_type,image_url,name_book,num_page,price,provider,publisher,size,url,year
0,Gege Akutami,Bìa mềm,https://salt.tikicdn.com/cache/280x280/ts/prod...,Chú Thuật Hồi Chiến: Tập 1,192,30000,NXB Kim Đồng,Nhà Xuất Bản Kim Đồng,113x176,https://tiki.vn/chu-thuat-hoi-chien-tap-1-p170...,2022
1,Tim Marshall,Bìa mềm,https://salt.tikicdn.com/cache/280x280/ts/prod...,Những Tù Nhân Của Địa Lý,430,210000,Nhã Nam,Nhà Xuất Bản Hội Nhà Văn,140x205,https://tiki.vn/nhung-tu-nhan-cua-dia-ly-p7378...,2020
2,Gege Akutami,Bìa mềm,https://salt.tikicdn.com/cache/280x280/ts/prod...,Chú Thuật Hồi Chiến Tập 2: Chú Thai Đới Thiên ...,192,30000,NXB Kim Đồng,Nhà Xuất Bản Kim Đồng,113x176,https://tiki.vn/chu-thuat-hoi-chien-tap-2-chu-...,2022
3,Morgan Housel,Bìa mềm,https://salt.tikicdn.com/cache/280x280/ts/prod...,Tâm Lý Học Về Tiền,384,189000,1980 Books,Nhà Xuất Bản Dân Trí,130x205,https://tiki.vn/tam-ly-hoc-ve-tien-p75953557.h...,2021
4,Trang Anh,Bìa mềm,https://salt.tikicdn.com/cache/280x280/ts/prod...,Tổng ôn ngữ pháp Tiếng Anh cô Trang Anh- Sách ...,606,179000,Aladanh,Nhà Xuất Bản Đại Học Quốc Gia Hà Nội,130x205,https://tiki.vn/tong-on-ngu-phap-tieng-anh-co-...,2021
...,...,...,...,...,...,...,...,...,...,...,...
15519,Naoko Takeuchi,Bìa mềm,https://salt.tikicdn.com/cache/280x280/ts/prod...,Anh Em Phi Hành Gia - Tập 26,192,45000,NXB Trẻ,NXB Trẻ,113x176,https://tiki.vn/anh-em-phi-hanh-gia-tap-26-p18...,2018
15520,Naoko Takeuchi,Bìa mềm,https://salt.tikicdn.com/cache/280x280/ts/prod...,HORIMIYA - Tập 14,192,50000,IPM,Nhà Xuất Bản Hồng Đức,113x176,https://tiki.vn/horimiya-tap-14-p179584488.htm...,2018
15521,Naoko Takeuchi,Bìa mềm,https://salt.tikicdn.com/cache/280x280/ts/prod...,Tân Doraemon Bóng Chày (Tập 3),192,16000,Nhà Xuất Bản Kim Đồng,Nhà Xuất Bản Hồng Đức,113x176,https://tiki.vn/tan-doraemon-bong-chay-tap-3-p...,2017
15522,Naoko Takeuchi,Bìa mềm,https://salt.tikicdn.com/cache/280x280/ts/prod...,Shin - Cậu Bé Bút Chì - Tập 27 (Tái Bản 2019),128,19000,Nhà Xuất Bản Kim Đồng,Nhà Xuất Bản Hồng Đức,145x205,https://tiki.vn/shin-cau-be-but-chi-tap-27-tai...,2019


In [156]:
for column in columns_df:
  train_name = np.concatenate((df[column].values.reshape(df[column].values.shape[0],1), np.ones((df[column].values.shape[0],1))), axis=1)
  columns = df.columns.copy().tolist()
  columns.remove('url')
  columns.remove('image_url')
  columns.remove(column)
  for i in columns:
    x = np.concatenate((df[i].values.reshape(df[i].values.shape[0],1), np.zeros((df[i].values.shape[0],1))), axis=1)
    train_name = np.concatenate((train_name, x), axis=0)
  df_1 = pd.DataFrame(train_name, columns = ['text','target'])
  df_1['text'] = df_1['text'].apply(lambda x:x.lower())
  df_1['text'] = df_1['text'].apply(lambda x:x.translate(str.maketrans(' ', ' ', string.punctuation)))


  # Get the V Freq
  V_freq = Counter(" ".join(df_1['text'].values.tolist()).split(" "))
  
  # Get the V
  V = len(V_freq.keys())   
  
  # get the freq_pos
  freq_pos = Counter(" ".join(df_1.loc[df_1.target==1]['text'].values.tolist()).split(" ")) 
  
  # get the freq_neg
  freq_neg = Counter(" ".join(df_1.loc[df_1.target==0]['text'].values.tolist()).split(" ")) 
  
  # get the number of positive and negative documents
  D_pos = sum(df_1.target==1)
  D_neg = sum(df_1.target==0)
  
  # get the number of unique positive and negative words
  N_pos = len(freq_pos.keys()) 
  N_neg = len(freq_neg.keys()) 
  
  logprior = np.log(D_pos/D_neg)

  df_2 = pd.read_csv('/content/drive/MyDrive/THDL/tiki1.csv')
  df_2 = df_2.drop(columns = ['Unnamed: 0'])
  df_2.astype(str)

  df_2['year'] = df_2['year'].apply(lambda x:str(x).split('.')[0])
  df_2['publisher'] = df_2['publisher'].apply(lambda x: str(x).replace('Nhà Xuất Bản ', ''))
  df_2['publisher'] = df_2['publisher'].apply(lambda x: str(x).replace('NXB ', ''))

  columns_df_2 = df_2.columns.tolist().copy()
  columns_df_2.remove('url')
  columns_df_2.remove('image_url')
  rate_max = 0
  column_max = ''
  for column_df_2 in columns_df_2:
    rate_sim = jarowinkler.similarity(column, column_df_2)
    #print(rate_sim)
    df_2_name = pd.DataFrame(df_2[column_df_2].astype(str))
    df_2_name['score'] = df_2_name[column_df_2].apply(lambda x:text_loglikelihood(x))
    df_2_name['prediction'] = df_2_name.score.apply(lambda x:int(x>0))
    if df_2_name.groupby(['prediction']).size()[0] == len(df_2_name.index): 
      if df_2_name.groupby(['prediction']).size().index[0] == 0:
        rate = 0
      if df_2_name.groupby(['prediction']).size().index[0] == 1:
        rate = 1
    else:
      rate = df_2_name.groupby(['prediction']).size()[1]/(df_2_name.groupby(['prediction']).size()[1]+df_2_name.groupby(['prediction']).size()[0])
    #print(rate)
    rate_total = 0.3*rate_sim + 0.7*rate
    #print(rate_total)
    if rate_total > rate_max:
      rate_max = rate_total
      column_max = column_df_2
    df_2_name = df_2_name[0:0] 
  print(column + ' of table newshop match to ' + column_max + ' of table tiki')
  print(rate_max)
  print('////////////////////////////')


author_name of table newshop match to author of table tiki
0.7052277891231067
////////////////////////////
cover_type of table newshop match to cover_type of table tiki
0.9093661427467148
////////////////////////////
name of table newshop match to name_book of table tiki
0.9463304131237653
////////////////////////////
num_page of table newshop match to num_page of table tiki
0.9167160525637721
////////////////////////////
price of table newshop match to price of table tiki
0.914371296057717
////////////////////////////
provider of table newshop match to provider of table tiki
0.5406080906982736
////////////////////////////
publisher of table newshop match to publisher of table tiki
0.49646354032465856
////////////////////////////
release_year of table newshop match to year of table tiki
0.765751953963755
////////////////////////////
size of table newshop match to size of table tiki
0.9869234733316155
////////////////////////////


In [175]:
for column in columns_df:
  train_name = np.concatenate((df[column].values.reshape(df[column].values.shape[0],1), np.ones((df[column].values.shape[0],1))), axis=1)
  columns = df.columns.copy().tolist()
  columns.remove('url')
  columns.remove('image_url')
  columns.remove(column)
  for i in columns:
    x = np.concatenate((df[i].values.reshape(df[i].values.shape[0],1), np.zeros((df[i].values.shape[0],1))), axis=1)
    train_name = np.concatenate((train_name, x), axis=0)
  df_1 = pd.DataFrame(train_name, columns = ['text','target'])
  df_1['text'] = df_1['text'].apply(lambda x:x.lower())
  df_1['text'] = df_1['text'].apply(lambda x:x.translate(str.maketrans(' ', ' ', string.punctuation)))


  # Get the V Freq
  V_freq = Counter(" ".join(df_1['text'].values.tolist()).split(" "))
  
  # Get the V
  V = len(V_freq.keys())   
  
  # get the freq_pos
  freq_pos = Counter(" ".join(df_1.loc[df_1.target==1]['text'].values.tolist()).split(" ")) 
  
  # get the freq_neg
  freq_neg = Counter(" ".join(df_1.loc[df_1.target==0]['text'].values.tolist()).split(" ")) 
  
  # get the number of positive and negative documents
  D_pos = sum(df_1.target==1)
  D_neg = sum(df_1.target==0)
  
  # get the number of unique positive and negative words
  N_pos = len(freq_pos.keys()) 
  N_neg = len(freq_neg.keys()) 
  
  logprior = np.log(D_pos/D_neg)

  df_2 = pd.read_csv('/content/drive/MyDrive/THDL/book3651.csv')
  df_2 = df_2.drop(columns = ['Unnamed: 0'])
  df_2.astype(str)
  #df_2['sale_price'] = df_2['sale_price'].apply(lambda x: str(x).split('.')[0])

  #df_2['year'] = df_2['year'].apply(lambda x:str(x).split('.')[0])
  df_2['publisher'] = df_2['publisher'].apply(lambda x: str(x).replace('Nhà Xuất Bản ', ''))
  df_2['publisher'] = df_2['publisher'].apply(lambda x: str(x).replace('NXB ', ''))
  df_2['provider'] = df_2['provider'].apply(lambda x: str(x).replace('Nhà Xuất Bản ', ''))
  df_2['provider'] = df_2['provider'].apply(lambda x: str(x).replace('NXB ', ''))
  columns_df_2 = df_2.columns.tolist().copy()
  columns_df_2.remove('url')
  #columns_df_2.remove('image_url')
  rate_max = 0
  column_max = ''
  for column_df_2 in columns_df_2:
    rate_sim = jarowinkler.similarity(column, column_df_2)
    #print(rate_sim)
    df_2_name = pd.DataFrame(df_2[column_df_2].astype(str))
    df_2_name['score'] = df_2_name[column_df_2].apply(lambda x:text_loglikelihood(x))
    df_2_name['prediction'] = df_2_name.score.apply(lambda x:int(x>0))
    if df_2_name.groupby(['prediction']).size()[0] == len(df_2_name.index): 
      if df_2_name.groupby(['prediction']).size().index[0] == 0:
        rate = 0
      if df_2_name.groupby(['prediction']).size().index[0] == 1:
        rate = 1
    else:
      rate = df_2_name.groupby(['prediction']).size()[1]/(df_2_name.groupby(['prediction']).size()[1]+df_2_name.groupby(['prediction']).size()[0])
    #print(rate)
    rate_total = 0.3*rate_sim + 0.7*rate
    #print(rate_total)
    if rate_total > rate_max:
      rate_max = rate_total
      column_max = column_df_2
    df_2_name = df_2_name[0:0] 
  print(column + ' of table newshop match to ' + column_max + ' of table fahasa')
  print(rate_max)
  print('////////////////////////////')


author_name of table newshop match to author_name of table fahasa
0.6880057803468208
////////////////////////////
cover_type of table newshop match to cover_type of table fahasa
0.9906069364161849
////////////////////////////
name of table newshop match to name of table fahasa
0.9754335260115605
////////////////////////////
num_page of table newshop match to num_page of table fahasa
0.9028901734104045
////////////////////////////
price of table newshop match to price of table fahasa
0.3004335260115607
////////////////////////////
provider of table newshop match to provider of table fahasa
0.5527456647398843
////////////////////////////
publisher of table newshop match to publisher of table fahasa
0.45173410404624276
////////////////////////////
release_year of table newshop match to release_year of table fahasa
0.3
////////////////////////////
size of table newshop match to size of table fahasa
0.3
////////////////////////////


In [177]:
for column in columns_df:
  train_name = np.concatenate((df[column].values.reshape(df[column].values.shape[0],1), np.ones((df[column].values.shape[0],1))), axis=1)
  columns = df.columns.copy().tolist()
  columns.remove('url')
  columns.remove('image_url')
  columns.remove(column)
  for i in columns:
    x = np.concatenate((df[i].values.reshape(df[i].values.shape[0],1), np.zeros((df[i].values.shape[0],1))), axis=1)
    train_name = np.concatenate((train_name, x), axis=0)
  df_1 = pd.DataFrame(train_name, columns = ['text','target'])
  df_1['text'] = df_1['text'].apply(lambda x:x.lower())
  df_1['text'] = df_1['text'].apply(lambda x:x.translate(str.maketrans(' ', ' ', string.punctuation)))


  # Get the V Freq
  V_freq = Counter(" ".join(df_1['text'].values.tolist()).split(" "))
  
  # Get the V
  V = len(V_freq.keys())   
  
  # get the freq_pos
  freq_pos = Counter(" ".join(df_1.loc[df_1.target==1]['text'].values.tolist()).split(" ")) 
  
  # get the freq_neg
  freq_neg = Counter(" ".join(df_1.loc[df_1.target==0]['text'].values.tolist()).split(" ")) 
  
  # get the number of positive and negative documents
  D_pos = sum(df_1.target==1)
  D_neg = sum(df_1.target==0)
  
  # get the number of unique positive and negative words
  N_pos = len(freq_pos.keys()) 
  N_neg = len(freq_neg.keys()) 
  
  logprior = np.log(D_pos/D_neg)

  df_2 = pd.read_csv('/content/drive/MyDrive/THDL/fahasa1.csv')
  df_2 = df_2.drop(columns = ['Unnamed: 0'])
  df_2.astype(str)
  df_2['sale_price'] = df_2['sale_price'].apply(lambda x: str(x).split('.')[0])

  #df_2['year'] = df_2['year'].apply(lambda x:str(x).split('.')[0])
  df_2['publisher'] = df_2['publisher'].apply(lambda x: str(x).replace('Nhà Xuất Bản ', ''))
  df_2['publisher'] = df_2['publisher'].apply(lambda x: str(x).replace('NXB ', ''))
  df_2['provider'] = df_2['provider'].apply(lambda x: str(x).replace('Nhà Xuất Bản ', ''))
  df_2['provider'] = df_2['provider'].apply(lambda x: str(x).replace('NXB ', ''))
  columns_df_2 = df_2.columns.tolist().copy()
  columns_df_2.remove('url')
  columns_df_2.remove('image_url')
  rate_max = 0
  column_max = ''
  for column_df_2 in columns_df_2:
    rate_sim = jarowinkler.similarity(column, column_df_2)
    #print(rate_sim)
    df_2_name = pd.DataFrame(df_2[column_df_2].astype(str))
    df_2_name['score'] = df_2_name[column_df_2].apply(lambda x:text_loglikelihood(x))
    df_2_name['prediction'] = df_2_name.score.apply(lambda x:int(x>0))
    if df_2_name.groupby(['prediction']).size()[0] == len(df_2_name.index): 
      if df_2_name.groupby(['prediction']).size().index[0] == 0:
        rate = 0
      if df_2_name.groupby(['prediction']).size().index[0] == 1:
        rate = 1
    else:
      rate = df_2_name.groupby(['prediction']).size()[1]/(df_2_name.groupby(['prediction']).size()[1]+df_2_name.groupby(['prediction']).size()[0])
    #print(rate)
    rate_total = 0.3*rate_sim + 0.7*rate
    #print(rate_total)
    if rate_total > rate_max:
      rate_max = rate_total
      column_max = column_df_2
    df_2_name = df_2_name[0:0] 
  print(column + ' of table newshop match to ' + column_max + ' of table fahasa')
  print(rate_max)
  print('////////////////////////////')


author_name of table newshop match to author_name of table fahasa
0.8375637972099352
////////////////////////////
cover_type of table newshop match to cover_type of table fahasa
0.9938074174889417
////////////////////////////
name of table newshop match to name of table fahasa
0.996189179993195
////////////////////////////
num_page of table newshop match to num_page of table fahasa
0.909254848587955
////////////////////////////
price of table newshop match to sale_price of table fahasa
0.7992752636951344
////////////////////////////
provider of table newshop match to provider of table fahasa
0.3190541000340252
////////////////////////////
publisher of table newshop match to publisher of table fahasa
0.5231711466485198
////////////////////////////
release_year of table newshop match to release_year of table fahasa
0.9068730860837018
////////////////////////////
size of table newshop match to size of table fahasa
0.3
////////////////////////////


In [172]:
df_2 = pd.read_csv('/content/drive/MyDrive/THDL/book3651.csv')
df_2 = df_2.drop(columns = ['Unnamed: 0'])
df_2.astype(str)

,author_name,cover_type,name,num_page,price,provider,publisher,release_year,size,url
0,Aesop,Bìa cứng,Truyện Ngụ Ngôn E Dốp,88,98.000,Liên Việt Books,Văn học,2017.0,"19x26,5 (cm)",https://book365.vn/sach/19118_truyen-ngu-ngon-...
1,"Joanne Taylore-Knowles, Steve Taylore-Knowles,...",Bìa mềm,OpenMind 1: Student Book,88,100.000,Công ty sách CDIMEX,MacMillan,2010.0,(cm),https://book365.vn/sach/1144241_openmind-1-stu...
2,Murray Eldridge,Bìa mềm,Hiệu suất đỉnh cao,304,130.000,Saigon Books,Thế Giới,2019.0,20x15x3 (cm),https://book365.vn/sach/576262_hieu-suat-dinh-...
3,Alan Skinner,Bìa mềm,Sự Thật Về Hòn Đá Phù Thủy,416,98.000,FIRST NEWS,NXB Tổng Hợp TPHCM,2019.0,25x20x4 (cm),https://book365.vn/sach/1161580_su-that-ve-hon...
4,Vũ Minh Khương,Bìa mềm,"Hãy trỗi dậy, Việt Nam! Let's rise, Vietnam!",588,190.000,Nhà xuất bản Tri thức,Tri thức,2020.0,(cm),https://book365.vn/sach/450547_hay-troi-day-vi...
...,...,...,...,...,...,...,...,...,...,...
4839,HT Thích Minh Thiền,Bìa mềm,Tham thiền,302,189.000,Thái Hà Books,Tiếp tục,2020.0,(cm),https://book365.vn/sach/411073_tham-thien.html
4840,TS. Đặng Thị Việt Đức,Bìa mềm,Quản trị tri thức trong Doanh nghiệp - Thời hạ...,302,11962.5,NXB Thông tin và Truyền thông,Thông tin và Truyền thông,2017.0,(cm),https://book365.vn/sach/416559_quan-tri-tri-th...
4841,Phòng Thương mại quốc tế ICC,Bìa mềm,Bộ tập quán quốc tế về L/C - Các văn bản có hi...,302,9570,NXB Thông tin và Truyền thông,Thông tin và Truyền thông,2016.0,(cm),https://book365.vn/sach/416593_bo-tap-quan-quo...
4842,Lê Thị Thu Hà,Bìa mềm,Bảo hộ chỉ dẫn địa lý của Việt Nam trong điều ...,302,18183,NXB Thông tin và Truyền thông,Thông tin và Truyền thông,2017.0,(cm),https://book365.vn/sach/416497_bao-ho-chi-dan-...


In [169]:
df_2

,author_name,cover_type,image_url,name,num_page,provider,publisher,release_year,size,url,sale_price
0,Negi Haruba,Bìa Mềm,https://cdn0.fahasa.com/media/catalog/product/...,Nhà Có 5 Nàng Dâu - Tập 13,192,Nhà Xuất Bản Kim Đồng,NXB Kim Đồng,2022,17.6 x 11.3 cm,https://www.fahasa.com/nha-co-5-nang-dau-tap-1...,25000
1,ADACHI Toka,Bìa Mềm,https://cdn0.fahasa.com/media/catalog/product/...,Vị Thần Lang Thang Tập 22 - Tặng Kèm Postcard,190,Nhà Xuất Bản Kim Đồng,NXB Kim Đồng,2022,17.6 x 11.3 cm,https://www.fahasa.com/vi-than-lang-thang-tap-...,20000
2,Tomohito Oda,Bìa Mềm,https://cdn0.fahasa.com/media/catalog/product/...,Komi - Nữ Thần Sợ Giao Tiếp - Tập 8 - Tặng Kèm...,192,Nhà Xuất Bản Kim Đồng,NXB Kim Đồng,2022,17.6 x 11.3 cm,https://www.fahasa.com/komi-nu-than-so-giao-ti...,25000
3,Sho Aimoto,Bìa Mềm,https://cdn0.fahasa.com/media/catalog/product/...,Văn Phòng Thám Tử Quái Vật - Tập 3,192,NXB Trẻ,NXB Trẻ,2022,17.6 x 11.3 cm,https://www.fahasa.com/van-phong-tham-tu-quai-...,30000
4,Koyoharu Gotouge,Bìa Mềm,https://cdn0.fahasa.com/media/catalog/product/...,Thanh Gươm Diệt Quỷ - Kimetsu No Yaiba - Tập 1...,192,Nhà Xuất Bản Kim Đồng,NXB Kim Đồng,2020,17.6 x 11.3 cm,https://www.fahasa.com/thanh-guom-diet-quy-kim...,25000
...,...,...,...,...,...,...,...,...,...,...,...
2934,Đỗ Nhung,Bìa Mềm,https://cdn0.fahasa.com/media/catalog/product/...,Tổng Hợp Kiến Thức Và Bài Tập Tiếng Anh 6 (2020),200,Cấp 2,NXB Hồng Đức,2020,24 x 17 cm,https://www.fahasa.com/tong-hop-kien-thuc-va-b...,149000
2935,Đại Lợi,Bìa Mềm,https://cdn0.fahasa.com/media/catalog/product/...,Bài Tập Cuối Tuần Tiếng Anh Lớp 3 Tập 1 - Có Đ...,400,Cấp 1,NXB Đại Học Quốc Gia Hà Nội,2020,27 x 19 cm,https://www.fahasa.com/bai-tap-cuoi-tuan-tieng...,68000
2936,Maru Nagao,Bìa Mềm,https://cdn0.fahasa.com/media/catalog/product/...,Họa Sư Ma Mèo - Tập 13,208,Nhà Xuất Bản Kim Đồng,NXB Kim Đồng,2020,17.6 x 11.3 cm,https://www.fahasa.com/hoa-su-ma-meo-tap-13.html,28000
2937,Nhiều Tác Giả,Bìa Cứng,https://cdn0.fahasa.com/media/catalog/product/...,Truyện Kiều Tự Kể,140,Nhà Xuất Bản Kim Đồng,NXB Kim Đồng,2020,28 x 20 cm,https://www.fahasa.com/truyen-kieu-tu-ke.html,190000


In [53]:
df_1 = pd.DataFrame(train_name, columns = ['text','target'])
df_1

,text,target
0,Sách Tương Phản - Đồ Dùng Của Bé,1.0
1,Combo Sách Tương Phản (Bộ 4 Cuốn),1.0
2,Chạm - Hành Trình Cảm Xúc Của 12 Chòm Sao,1.0
3,Môn Đồ - Sư Phụ Giả - Tập 7,1.0
4,Ikigami - Tuyển Tập Những Câu Chuyện Lay Động ...,1.0
...,...,...
230638,145x20,0.0
230639,170x230,0.0
230640,135x20,0.0
230641,155x240,0.0


In [54]:
df_1['text'] = df_1['text'].apply(lambda x:x.lower())
df_1['text'] = df_1['text'].apply(lambda x:x.translate(str.maketrans(' ', ' ', string.punctuation)))
df_1

,text,target
0,sách tương phản đồ dùng của bé,1.0
1,combo sách tương phản bộ 4 cuốn,1.0
2,chạm hành trình cảm xúc của 12 chòm sao,1.0
3,môn đồ sư phụ giả tập 7,1.0
4,ikigami tuyển tập những câu chuyện lay động l...,1.0
...,...,...
230638,145x20,0.0
230639,170x230,0.0
230640,135x20,0.0
230641,155x240,0.0


In [55]:
# Get the V Freq
V_freq = Counter(" ".join(df_1['text'].values.tolist()).split(" "))
 
# Get the V
V = len(V_freq.keys())   
 
# get the freq_pos
freq_pos = Counter(" ".join(df_1.loc[df_1.target==1]['text'].values.tolist()).split(" ")) 
 
# get the freq_neg
freq_neg = Counter(" ".join(df_1.loc[df_1.target==0]['text'].values.tolist()).split(" ")) 
 
# get the number of positive and negative documents
D_pos = sum(df_1.target==1)
D_neg = sum(df_1.target==0)
 
# get the number of unique positive and negative words
N_pos = len(freq_pos.keys()) 
N_neg = len(freq_neg.keys()) 
 
logprior = np.log(D_pos/D_neg)
 



In [56]:
df_1['score'] = df_1.text.apply(lambda x:text_loglikelihood(x))
df_1['prediction'] = df_1.score.apply(lambda x:int(x>0))
 
# confusion matrix
df_1.groupby(['target','prediction']).size().reset_index()

,target,prediction,0
0,0.0,0,203915
1,0.0,1,1101
2,1.0,0,394
3,1.0,1,25233


In [110]:
df_2 = pd.read_csv('/content/drive/MyDrive/THDL/tiki1.csv')
df_2 = df_2.drop(columns = ['Unnamed: 0'])
df_2.astype(str)

df_2

,author,cover_type,image_url,name_book,num_page,price,provider,publisher,size,url,year
0,Gege Akutami,Bìa mềm,https://salt.tikicdn.com/cache/280x280/ts/prod...,Chú Thuật Hồi Chiến: Tập 1,192,30000,NXB Kim Đồng,Nhà Xuất Bản Kim Đồng,113x176,https://tiki.vn/chu-thuat-hoi-chien-tap-1-p170...,2022.0
1,Tim Marshall,Bìa mềm,https://salt.tikicdn.com/cache/280x280/ts/prod...,Những Tù Nhân Của Địa Lý,430,210000,Nhã Nam,Nhà Xuất Bản Hội Nhà Văn,140x205,https://tiki.vn/nhung-tu-nhan-cua-dia-ly-p7378...,2020.0
2,Gege Akutami,Bìa mềm,https://salt.tikicdn.com/cache/280x280/ts/prod...,Chú Thuật Hồi Chiến Tập 2: Chú Thai Đới Thiên ...,192,30000,NXB Kim Đồng,Nhà Xuất Bản Kim Đồng,113x176,https://tiki.vn/chu-thuat-hoi-chien-tap-2-chu-...,2022.0
3,Morgan Housel,Bìa mềm,https://salt.tikicdn.com/cache/280x280/ts/prod...,Tâm Lý Học Về Tiền,384,189000,1980 Books,Nhà Xuất Bản Dân Trí,130x205,https://tiki.vn/tam-ly-hoc-ve-tien-p75953557.h...,2021.0
4,Trang Anh,Bìa mềm,https://salt.tikicdn.com/cache/280x280/ts/prod...,Tổng ôn ngữ pháp Tiếng Anh cô Trang Anh- Sách ...,606,179000,Aladanh,Nhà Xuất Bản Đại Học Quốc Gia Hà Nội,130x205,https://tiki.vn/tong-on-ngu-phap-tieng-anh-co-...,2021.0
...,...,...,...,...,...,...,...,...,...,...,...
15519,Naoko Takeuchi,Bìa mềm,https://salt.tikicdn.com/cache/280x280/ts/prod...,Anh Em Phi Hành Gia - Tập 26,192,45000,NXB Trẻ,NXB Trẻ,113x176,https://tiki.vn/anh-em-phi-hanh-gia-tap-26-p18...,2018.0
15520,Naoko Takeuchi,Bìa mềm,https://salt.tikicdn.com/cache/280x280/ts/prod...,HORIMIYA - Tập 14,192,50000,IPM,Nhà Xuất Bản Hồng Đức,113x176,https://tiki.vn/horimiya-tap-14-p179584488.htm...,2018.0
15521,Naoko Takeuchi,Bìa mềm,https://salt.tikicdn.com/cache/280x280/ts/prod...,Tân Doraemon Bóng Chày (Tập 3),192,16000,Nhà Xuất Bản Kim Đồng,Nhà Xuất Bản Hồng Đức,113x176,https://tiki.vn/tan-doraemon-bong-chay-tap-3-p...,2017.0
15522,Naoko Takeuchi,Bìa mềm,https://salt.tikicdn.com/cache/280x280/ts/prod...,Shin - Cậu Bé Bút Chì - Tập 27 (Tái Bản 2019),128,19000,Nhà Xuất Bản Kim Đồng,Nhà Xuất Bản Hồng Đức,145x205,https://tiki.vn/shin-cau-be-but-chi-tap-27-tai...,2019.0


In [102]:
df_2_name['score'] = df_2_name['sale_price'].apply(lambda x:text_loglikelihood(x))
df_2_name['prediction'] = df_2_name.score.apply(lambda x:int(x>0))
df_2_name

,sale_price,score,prediction
0,25000.0,-2.079442,0
1,20000.0,-2.079442,0
2,25000.0,-2.079442,0
3,30000.0,-2.079442,0
4,25000.0,-2.079442,0
...,...,...,...
2934,149000.0,-2.079442,0
2935,68000.0,-2.079442,0
2936,28000.0,-2.079442,0
2937,190000.0,-2.079442,0


In [70]:
if df_2_name.groupby(['prediction']).size()[0] == len(df_2_name.index): 
  rate = 1
else:
  rate = df_2_name.groupby(['prediction']).size()[1]/(df_2_name.groupby(['prediction']).size()[1]+df_2_name.groupby(['prediction']).size()[0])
print(rate)

0.0013610071452875127


In [203]:
df_1 = pd.read_csv('/content/drive/MyDrive/THDL/newshop1.csv')
df_1 = df_1.drop(columns = ['Unnamed: 0'])
df_2 = pd.read_csv('/content/drive/MyDrive/THDL/tiki1.csv')
df_2 = df_2.drop(columns = ['Unnamed: 0'])
df_3 = pd.read_csv('/content/drive/MyDrive/THDL/fahasa1.csv')
df_3 = df_3.drop(columns = ['Unnamed: 0'])
df_4 = pd.read_csv('/content/drive/MyDrive/THDL/book3651.csv')
df_4 = df_4.drop(columns = ['Unnamed: 0'])
df_4['image_url'] = ['No image']*df_4.shape[0]


df_1 = pd.concat([df_1,df_2.rename(columns={'author':'author_name', 'name_book':'name', 'year':'release_year'}), 
                  df_3.rename(columns={'sale_price':'price'}), df_4], ignore_index=True)



25627
15524


In [202]:
df_1

,author_name,cover_type,image_url,name,num_page,price,provider,publisher,release_year,size,url
0,Fuji,Mềm,https://newshop.vn/public/uploads/products/485...,Sách Tương Phản - Đồ Dùng Của Bé,20,60000,Nhà Sách AZ Việt Nam,Thế Giới,2022,150x150,https://newshop.vn/sach-tuong-phan-do-dung-cua...
1,Fuji,Cứng,https://newshop.vn/public/uploads/products/485...,Combo Sách Tương Phản (Bộ 4 Cuốn),20,240000,Nhà Sách AZ Việt Nam,Thế Giới,2022,150x150,https://newshop.vn/combo-sach-tuong-phan-bo-4-...
2,Mèo Vũ Trụ,Mềm,https://newshop.vn/public/uploads/products/485...,Chạm - Hành Trình Cảm Xúc Của 12 Chòm Sao,184,99000,Nhà Sách AZ Việt Nam,Thế Giới,2022,130x180,https://newshop.vn/cham-hanh-trinh-cam-xuc-cua...
3,Manh A Công Tử,Mềm,https://newshop.vn/public/uploads/products/485...,Môn Đồ - Sư Phụ Giả - Tập 7,224,129000,Nhà Sách AZ Việt Nam,Thế Giới,2022,145x205,https://newshop.vn/mon-do-su-phu-gia-tap-7.htm...
4,Motoro Mase,Mềm,https://newshop.vn/public/uploads/products/485...,Ikigami - Tuyển Tập Những Câu Chuyện Lay Động ...,216,45000,NXB Trẻ,Trẻ,2022,130x10,https://newshop.vn/ikigami-tuyen-tap-nhung-cau...
...,...,...,...,...,...,...,...,...,...,...,...
48929,HT Thích Minh Thiền,Bìa mềm,No image,Tham thiền,302,189.000,Thái Hà Books,Tiếp tục,2020.0,(cm),https://book365.vn/sach/411073_tham-thien.html
48930,TS. Đặng Thị Việt Đức,Bìa mềm,No image,Quản trị tri thức trong Doanh nghiệp - Thời hạ...,302,11962.5,NXB Thông tin và Truyền thông,Thông tin và Truyền thông,2017.0,(cm),https://book365.vn/sach/416559_quan-tri-tri-th...
48931,Phòng Thương mại quốc tế ICC,Bìa mềm,No image,Bộ tập quán quốc tế về L/C - Các văn bản có hi...,302,9570,NXB Thông tin và Truyền thông,Thông tin và Truyền thông,2016.0,(cm),https://book365.vn/sach/416593_bo-tap-quan-quo...
48932,Lê Thị Thu Hà,Bìa mềm,No image,Bảo hộ chỉ dẫn địa lý của Việt Nam trong điều ...,302,18183,NXB Thông tin và Truyền thông,Thông tin và Truyền thông,2017.0,(cm),https://book365.vn/sach/416497_bao-ho-chi-dan-...


In [205]:
df_1.to_csv('schema_matching_data.csv')